# Boosting para la clasificación

En este notebook planteamos un modelo de boosting para la predicción de la supervivencia en el Titanic. El boosting emplea árboles que se generan de manera secuencial para ir reduciendo el error incrementalmente. Entrenaremo los modelos y calcularemos como siempre las métricas propias de la clasificación. En este notebook emplearemos los siguientes módulos:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

Cargamos los datos de entrenamiento y validación:

In [ ]:
X_train = pd.read_csv('./data/xtrain_tit.csv')
X_test = pd.read_csv('./data/xtest_tit.csv')
y_train = pd.read_csv('./data/ytrain_tit.csv')
y_test = pd.read_csv('./data/ytest_tit.csv')

Comprobamos que los datos se hayan cargado de manera adecuada:

In [ ]:
X_train.head()

In [ ]:
X_train.shape

In [ ]:
X_test.head()

In [ ]:
X_test.shape

## Construcción del modelo

Comenzamos como siempre instanciando el modelo:

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
boost_classifier = GradientBoostingClassifier()

Comenzamos instanciándolo con los parámetros predeterminados y ajustándolo a los datos:

In [ ]:
boost_classifier.fit(X_train, y_train)

Calculamos la precisión en entrenamiento y validación:

In [ ]:
boost_classifier.score(X_train, y_train)

In [ ]:
boost_classifier.score(X_test, y_test)

En este caso no se ha producido overfitting. Podemos intentar mejorar algo la métrica:

In [ ]:
boost_classifier_tuned = GradientBoostingClassifier(learning_rate=0.075)

In [ ]:
boost_classifier_tuned.fit(X_train, y_train)

In [ ]:
boost_classifier_tuned.score(X_train, y_train)

In [ ]:
boost_classifier_tuned.score(X_test, y_test)

Hemos conseguido llegar prácticamente al 85% en validación. Un resultado nada desdeñable. Calculamos las métricas propias de la clasificación para verficiar que nuestro clasificador está equilibrado:

In [ ]:
y_boost_pred = boost_classifier_tuned.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_boost_pred)
print(confusion_matrix)

La matriz de confusión nos muestra que está bastante equilibrado. Podemos calcular las métricas específicas:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_boost_pred))

Por último podemos comparar la curva ROC para verificar la capacidad de generalización de nuestro modelo independientemente del umbral elegido:

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, boost_classifier_tuned.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, boost_classifier_tuned.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Boosting (Área bajo la curva = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Ratio de falsos positivos')
plt.ylabel('Ratio de verdaderos positivos')
plt.title('Curva ROC')
plt.legend(loc="lower right")
#plt.savefig('Log_ROC') si descomentas esta línea puedes guardar la gráfica
plt.show()

Por último podemos comprobar que las variables más relevantes coinciden con las observadas en los anteriores modelos de clasificación: edad, sexo y si van o no en tercera clase.

In [ ]:
importancia_predictores = pd.DataFrame(
                            {'Variable predictora': X_train.columns,
                             'Importancia': boost_classifier_tuned.feature_importances_}
                            )
importancia_predictores.sort_values('Importancia', ascending=False)